## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [151]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [152]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [154]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [155]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [156]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json') as inf:
    query1 = json.load(inf)

In [157]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_negatively_regulates_entity"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Gene"
                    ]
                }
            }
        }
    }
}


In [158]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=0798f6c5-c2ac-4dc8-ae1f-febd208dc1a4


In [159]:
retrieve_ars_results(kcresult)

Done
Done
kp-icees Done 0
Done
kp-textmining Done 0
Done
kp-genetics Done 0
Done
kp-molecular Done 0
Done
ara-improving Done 4
Done
kp-chp Done 0
Error
kp-cam Error 0
Done
ara-aragorn Done 31
Done
ara-unsecret Done 0
Done
ara-bte Done 8
Unknown
ara-ncats Unknown 0
Done
ara-arax Done 3
Error
ara-aragorn-exp Error 0
Error
kp-cohd Error 0
Error
ara-explanatory Error 0
Done
kp-icees-dili Done 0
Done
kp-openpredict Done 0
Error
ara-robokop Error 0


{'pk_id': 'https://arax.ncats.io/?source=ARS&id=0798f6c5-c2ac-4dc8-ae1f-febd208dc1a4',
 'kp-icees': 'No Results',
 'kp-textmining': 'No Results',
 'kp-genetics': 'No Results',
 'kp-molecular': 'No Results',
 'ara-improving': 'Results',
 'kp-chp': 'No Results',
 'kp-cam': 'Error',
 'ara-aragorn': 'Results',
 'ara-unsecret': 'No Results',
 'ara-bte': 'Results',
 'ara-arax': 'Results',
 'ara-aragorn-exp': 'Error',
 'kp-cohd': 'Error',
 'ara-explanatory': 'Error',
 'kp-icees-dili': 'No Results',
 'kp-openpredict': 'No Results',
 'ara-robokop': 'Error'}

In [243]:
kcresult

'0798f6c5-c2ac-4dc8-ae1f-febd208dc1a4'

In [160]:
kcresult = '0798f6c5-c2ac-4dc8-ae1f-febd208dc1a4'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

In [161]:
j.keys()

dict_keys(['message', 'status', 'actor', 'children'])

In [162]:
j

{'message': '0798f6c5-c2ac-4dc8-ae1f-febd208dc1a4',
 'status': 'Done',
 'actor': {'pk': 9,
  'channel': 'general',
  'agent': 'ars-default-agent',
  'path': ''},
 'children': [{'message': '04260b76-5aaf-4331-a0c2-82483bc3068e',
   'status': 'Done',
   'code': '200',
   'actor': {'pk': 18,
    'channel': 'general',
    'agent': 'kp-icees',
    'path': 'runquery'},
   'children': []},
  {'message': '05f4a9f5-117b-4f08-85e2-b52c2ab09983',
   'status': 'Done',
   'code': '200',
   'actor': {'pk': 13,
    'channel': 'general',
    'agent': 'kp-textmining',
    'path': 'runquery'},
   'children': []},
  {'message': '1a6ad828-c2e0-45c6-95b2-b7ab7f9e7c5d',
   'status': 'Done',
   'code': '200',
   'actor': {'pk': 7,
    'channel': 'general',
    'agent': 'kp-genetics',
    'path': 'runquery'},
   'children': []},
  {'message': '24a2483c-c21a-4ed3-9d69-6649c078b465',
   'status': 'Done',
   'code': '200',
   'actor': {'pk': 8,
    'channel': 'general',
    'agent': 'kp-molecular',
    'path': '

In [145]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [175]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [176]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1a_DILI-three-hop-from-MONDO:0005359_DILI.json,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
4,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL165,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL116438,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
9,B.3a_DILI-fourth-one-hop-from-CHEBI:41879_Dexa...,NaN,NaN,NaN,NaN,NaN,Note that this query should return numerous re...


In [183]:
check_sheet.iloc[0][3]

'n1'

In [242]:
dictionary_test = {}
for child in j['children']:
    print(child['actor']['agent'])
    dictionary_test
    if child['status']  == 'Done':
        #print(child['actor']['agent'])
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        #print(child_response['fields']['data']['message']['knowledge_graph']['edges'])
        if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
            test_att_values =[]
            
            node_num = check_sheet.iloc[0][3]
            query_id = check_sheet.iloc[0][1]
             
            nodes = child_response['fields']['data']['message']['query_graph']['nodes']
            for val in child_response['fields']['data']['message']['results']:
                
                
                if query_id in val['node_bindings'][node_num][0]['id']:
                    print('id',': PASS')
                else:
                    print('id not find')
                
#                 att = list(edge_ex[val]['attributes'])
#                 #print(att)
#                 for tx in att:
#                     #print(tx)
#                     if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
#                         #print(tx['value'])
#                         x = tx['value']
                        
#                         test_att_values.append(x)
#                         test_att = set(flatten_list(test_att_values))
                        
#                         print(test_att_values)
#                         dictionary_test[child['actor']['agent']] = test_att
#         else:
#             pass
    

kp-icees
kp-textmining
kp-genetics
kp-molecular
ara-improving
id not find
id not find
id not find
id not find
kp-chp
kp-cam
ara-aragorn
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
ara-unsecret
ara-bte
id not find
id not find
id not find
id not find
id not find
id not find
id not find
id not find
ara-ncats
ara-arax
id not find
id not find
id not find
ara-aragorn-exp
kp-cohd
ara-explanatory
kp-icees-dili
kp-openpredict
ara-robokop


In [234]:
node_num

'n1'

In [149]:
dictionary_test

{'ara-aragorn': {'CTD',
  'DisGeNET',
  'JensenLab Knowledge GHR',
  'JensenLab Knowledge UniProtKB-KW',
  'JensenLab Text Mining',
  'Monarch',
  'eRAM',
  'infores:aragorn',
  'infores:aragorn-ranker-ara',
  'infores:automat-robokop',
  'infores:automat.cord19',
  'infores:automat.covidkop',
  'infores:automat.hetio',
  'infores:automat.mychem-info',
  'infores:automat.ontology-hierarchy',
  'infores:automat.pharos',
  'infores:automat.robokop',
  'infores:automat.uberongraph',
  'infores:automat:biolink',
  'infores:chembl',
  'infores:clinvar',
  'infores:cord19-scibite',
  'infores:diseases',
  'infores:disgenet',
  'infores:gencc',
  'infores:genetics-data-provider',
  'infores:gwas-catalog',
  'infores:hetio',
  'infores:mesh',
  'infores:mondo',
  'infores:ontological-hierarchy',
  'infores:pharos',
  'infores:rtx-kg2',
  'infores:semmeddb',
  'infores:spoke',
  'infores:sri-reference-kg',
  'infores:ubergraph'},
 'ara-arax': {'infores:arax',
  'infores:chembl',
  'infores:disg

In [ ]:
dictionary={}
results = {}
dictionary_2 = {}
for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    else:
                        pass 
                else:
                    dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)

In [ ]:
dictionary_2

In [ ]:
for child in j['children']:
    #print(child['status'])
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        if child_response['fields']['data']['message']['knowledge_graph']['edges']:
            print('correct')
        else:
            print('incorrect')

In [ ]:
child_response['fields']['data']['message']['knowledge_graph']

In [ ]:
mm = child_response['pk']
mm

In [ ]:
xt = f'{ars_url}/messages/{childmessage_id}/{mm}'
xt

In [ ]:
f'{ars_url}/model/{mm}'

In [ ]:
requests.get(xt).json

In [ ]:
childmessage_id = child['message']
child_url = f'{ars_url}/messages/{childmessage_id}'

 child_response = requests.get(child_url).json()

In [ ]:
result_status = retrieve_ars_results(kcresult)

In [ ]:
kcresult

In [ ]:
result_status 

In [ ]:
pwd

In [ ]:
df

In [ ]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [ ]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
df